In [1]:
import os
import time
from math import exp
import numpy as np
from IPython.display import display, HTML

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import openai
from openai import OpenAI
import json
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

# Assign pinecone PINECONE_API_KEY
pinecone_api_key = os.getenv("PINECONE_API_KEY")
#print(pinecone_api_key)

client = OpenAI()
pc = Pinecone(api_key=pinecone_api_key)

In [3]:
# Create your own vector database index name
index_name = 'cstugpt-dc'

embed_model = "text-embedding-3-small"

In [4]:
namespace = 'dc'
index = pc.Index(index_name)

In [5]:
def pinecone_upsert_chunk(text, rec_count, index_name, namespace):

    res = client.embeddings.create(input=text, model=embed_model)

    embed = res.data[0].embedding
    print("Embeds length:", len(embed))
    
    count = rec_count
    # Meta data preparation
    metadata = {'text': text}
    
    index.upsert(vectors=[{"id": namespace + '_' + str(count), "metadata": metadata, "values": embed}], 
                 namespace=namespace)

In [6]:
vector_db = index.describe_index_stats()
vector_db

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'dc': {'vector_count': 47}},
 'total_vector_count': 47}

In [7]:
next_count = int(vector_db['total_vector_count'] + 1)
next_count

48

In [8]:
# Upsert the chunk of info into database
#text = "Staff Reporters:\nCole Euken\nDon Zirbel\nJennifer James"
#text = "Leadership:\nJohn Mills(CEO & Co-Founder)\nDavid Merrit((CEO & Co-Founder)\nBrian Harris(CPO)\nNick Russell(VP of Operations)"
text = "Van Lam is 90 year old."
pinecone_upsert_chunk(text, next_count, index_name, namespace)

Embeds length: 1536
